1. TUTORIAL: https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi

2. MODELS: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


In [ ]:
!pip install tensorflow-gpu

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
print(device_name)
print("'sup!'")

!/usr/local/cuda/bin/nvcc --version

In [ ]:
#baixar a api
%cd /content/

!wget https://github.com/tensorflow/models/archive/refs/heads/master.zip  #baixar a API mais recete disponibilizada pela TensorFlow
!unzip master.zip
!shred -u -z -n 20 master.zip #deletar o arquivo

In [ ]:
#por indicações do tensor flow
%cd /content/models-master/research
# Compile protos.
!protoc ./object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp ./object_detection/packages/tf2/setup.py .
!python -m pip install .

"""
#instalar a api

%cd /content/models-master/research

#This creates a name_pb2.py file from every name.proto file in the \object_detection\protos folder
!protoc --python_out=. ./object_detection/protos/anchor_generator.proto ./object_detection/protos/argmax_matcher.proto ./object_detection/protos/bipartite_matcher.proto ./object_detection/protos/box_coder.proto ./object_detection/protos/box_predictor.proto ./object_detection/protos/eval.proto ./object_detection/protos/faster_rcnn.proto ./object_detection/protos/faster_rcnn_box_coder.proto ./object_detection/protos/grid_anchor_generator.proto ./object_detection/protos/hyperparams.proto ./object_detection/protos/image_resizer.proto ./object_detection/protos/input_reader.proto ./object_detection/protos/losses.proto ./object_detection/protos/matcher.proto ./object_detection/protos/mean_stddev_box_coder.proto ./object_detection/protos/model.proto ./object_detection/protos/optimizer.proto ./object_detection/protos/pipeline.proto ./object_detection/protos/post_processing.proto ./object_detection/protos/preprocessor.proto ./object_detection/protos/region_similarity_calculator.proto ./object_detection/protos/square_box_coder.proto ./object_detection/protos/ssd.proto ./object_detection/protos/ssd_anchor_generator.proto ./object_detection/protos/string_int_label_map.proto ./object_detection/protos/train.proto ./object_detection/protos/keypoint_box_coder.proto ./object_detection/protos/multiscale_anchor_generator.proto ./object_detection/protos/graph_rewriter.proto ./object_detection/protos/calibration.proto ./object_detection/protos/flexible_grid_anchor_generator.proto
object_detection/protos/ssd.proto --python_out=.
#na mesma pasta, research:
!python setup.py build
!python setup.py install
"""

In [ ]:
#ecolher o modelo a ser treinado
%cd /content/

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
#!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz #320x320
#!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz #300x300

!tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!shred -u -z -n 20 ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz #deletar o arquivo

In [ ]:
#copiar a configuração desse modelo da pasta samples da api
!cp /content/models-master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config /content/drive/MyDrive/blocos_UFRB/object-detection/training/


In [ ]:
#copiar train.py
!cp /content/models-master/research/object_detection/legacy/train.py /content/models-master/research/object_detection/

In [ ]:
%cd /content/models-master/research/object_detection/

!python model_main_tf2.py \
        --logtostderr \
        --model_dir="/content/drive/MyDrive/blocos_UFRB/object-detection/outputs_efficienttdet" \
        --pipeline_config_path=/content/drive/MyDrive/blocos_UFRB/object-detection/training/ssd_efficientdet_d0_512x512_coco17_tpu-8.config

**Exportar para pb**

In [ ]:
!pip install tf-nightly

In [ ]:
#exportar modelo
%cd /content/models-master/research/object_detection
!python export_tflite_graph_tf2.py \
 --pipeline_config_path=/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config \
 --trained_checkpoint_dir=/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ \
 --output_directory=/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/


In [ ]:
import tensorflow as tf

In [ ]:
model_dir = "/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model"
saved_model_dir = '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model'

In [ ]:
# Convert the model


model = tf.saved_model.load(model_dir)
concrete_func = model.signatures['serving_default']
concrete_func.inputs[0].set_shape([1, 320, 320, 3])
tf.saved_model.save(model, model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir, signature_keys=['serving_default'])
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

# Save the model.
with open('/content/efficientdet_d0_coco17_tpu-32/saved_model/model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('/content/model.tflite', 'wb') as f:
  f.write(tflite_model)